In [1]:
import pandas as pd
import numpy as np

from m4_feats_functions import *
from m5_models import *
from m7_utils import *
from m3_model_params import lgb_params_1, lgb_params_2, lgb_params_3, xgb_params
from sklearn.metrics import mean_squared_error
import pandas as pd

In [2]:
# Paths to the train and test directories
INPUT_DIR = 'kaggle/input/linking-writing-processes-to-writing-quality'
FEATURE_STORE = 'feature_store'
train_dir = 'feature_store/train'
test_dir = 'feature_store/test'

train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

seed = 42
n_repeats = 5
n_splits = 10
target_col = 'score'

In [4]:
train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_1.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_1.pkl')
train_feats = train_feats.merge(train_scores, on='id', how='left')
train_feats, test_feats = countvectorize_one_one(train_logs, test_logs, train_feats, test_feats)
_, oof_1, rmse, model1 = xgb_cv_pipeline(train_feats=train_feats, 
                                        test_feats=test_feats, 
                                        xgb_params=xgb_params, 
                                        seed=seed, 
                                        n_repeats=n_repeats, 
                                        n_splits=n_splits)

Iterations:   0%|          | 0/5 [00:00<?, ?it/s]/root/miniconda3/envs/lrp/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [00:42:00] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
Iterations: 100%|██████████| 5/5 [03:03<00:00, 36.65s/it]

XGB Average RMSE over 50 folds: 0.604448


In [5]:
train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_2.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_2.pkl')
train_feats = train_feats.merge(train_scores, on='id', how='left')
train_feats, test_feats = countvectorize_one_one(train_logs, test_logs, train_feats, test_feats)
_, oof_2, rmse, model1 = xgb_cv_pipeline(train_feats=train_feats, 
                                        test_feats=test_feats, 
                                        xgb_params=xgb_params, 
                                        seed=seed, 
                                        n_repeats=n_repeats, 
                                        n_splits=n_splits)

Iterations: 100%|██████████| 5/5 [02:36<00:00, 31.28s/it]

XGB Average RMSE over 50 folds: 0.603451


In [6]:
train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_1.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_1.pkl')
train_feats = train_feats.merge(train_scores, on='id', how='left')
train_feats, test_feats = countvectorize_one_one(train_logs, test_logs, train_feats, test_feats)
_, oof_3, rmse, model1 = cv_pipeline(train_feats, test_feats, lgb_params_1, lgb_params_1['boosting_type'])

100%|██████████| 3/3 [00:00<00:00, 1865.52it/s]


LGBM Average RMSE over 50 folds: 0.602250


In [9]:
train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_2.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_2.pkl')
train_feats = train_feats.merge(train_scores, on='id', how='left')
train_feats, test_feats = countvectorize_one_one(train_logs, test_logs, train_feats, test_feats)
_, oof_4, rmse, model1 = cv_pipeline(train_feats, test_feats, lgb_params_1, lgb_params_1['boosting_type'])

blend = pd.concat([oof_1, oof_2, oof_3, oof_4], axis=0)
blend_scores = blend.groupby(['id','score'])['prediction'].mean().reset_index()
blend_rmse = mean_squared_error(blend_scores['score'], blend_scores['prediction'], squared=False)
print(f'Blend RMSE {blend_rmse:.6f}')

100%|██████████| 3/3 [00:00<00:00, 3194.44it/s]


LGBM Average RMSE over 50 folds: 0.602618
Blend RMSE 0.601894


In [13]:
train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_1.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_1.pkl')
train_feats = train_feats.merge(train_scores, on='id', how='left')
train_feats, test_feats = countvectorize_one_one(train_logs, test_logs, train_feats, test_feats)
_, oof_5, rmse, model1 = cv_pipeline(train_feats, test_feats, lgb_params_2, lgb_params_1['boosting_type'])

100%|██████████| 3/3 [00:00<00:00, 2829.53it/s]


LGBM Average RMSE over 50 folds: 0.602758


In [14]:
train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_2.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_2.pkl')
train_feats = train_feats.merge(train_scores, on='id', how='left')
train_feats, test_feats = countvectorize_one_one(train_logs, test_logs, train_feats, test_feats)
_, oof_6, rmse, model1 = cv_pipeline(train_feats, test_feats, lgb_params_2, lgb_params_1['boosting_type'])

100%|██████████| 3/3 [00:00<00:00, 3048.19it/s]


LGBM Average RMSE over 50 folds: 0.602102


In [17]:
for col in train_feats:
    print(col)

id
event_id_max
up_time_max
action_time_max
action_time_min
action_time_mean
action_time_std
action_time_quantile
action_time_sem
action_time_sum
action_time_skew
action_time_kurt
activity_nunique
down_event_nunique
up_event_nunique
text_change_nunique
cursor_position_nunique
cursor_position_max
cursor_position_quantile
cursor_position_sem
cursor_position_mean
word_count_nunique
word_count_max
word_count_quantile
word_count_sem
word_count_mean
action_time_gap1_max
action_time_gap1_min
action_time_gap1_mean
action_time_gap1_std
action_time_gap1_quantile
action_time_gap1_sem
action_time_gap1_sum
action_time_gap1_skew
action_time_gap1_kurt
cursor_position_change1_max
cursor_position_change1_mean
cursor_position_change1_std
cursor_position_change1_quantile
cursor_position_change1_sem
cursor_position_change1_sum
cursor_position_change1_skew
cursor_position_change1_kurt
word_count_change1_max
word_count_change1_mean
word_count_change1_std
word_count_change1_quantile
word_count_change1_sem
wo

In [16]:
blend = pd.concat([oof_1, oof_2, oof_3, oof_4, oof_5, oof_6], axis=0)
blend_scores = blend.groupby(['id','score'])['prediction'].mean().reset_index()
blend_rmse = mean_squared_error(blend_scores['score'], blend_scores['prediction'], squared=False)
print(f'Blend RMSE {blend_rmse:.6f}')

Blend RMSE 0.601335


In [45]:
from sklearn.feature_extraction.text import CountVectorizer

def countvectorize_one_one(train_logs, test_logs, train_feats, test_feats):
    essays = getEssays(train_logs)
    c_vect = CountVectorizer(ngram_range=(1, 1))
    tr_toks = c_vect.fit_transform(essays['essay']).todense()
    tr_toks_df = pd.DataFrame(columns = [f'tok_{i}' for i in range(tr_toks.shape[1])], data=tr_toks)
    train_feats = pd.concat([train_feats, tr_toks_df], axis=1)

    test_essay = getEssays(test_logs)
    ts_toks = c_vect.fit_transform(test_essay['essay']).todense()
    ts_toks_df = pd.DataFrame(columns = [f'tok_{i}' for i in range(ts_toks.shape[1])], data=ts_toks)
    test_feats = pd.concat([test_feats, ts_toks_df], axis=1)
    return train_feats, test_feats

In [5]:
# TEST HYPERPARAMS from search .csv
import ast
params_df = pd.read_csv('params.csv')
params_df['params'] = params_df['params'].apply(ast.literal_eval)

for i in range(3):
    params = params_df.loc[i]['params']
    params.pop('verbose')
    params.pop('random_state')
    params['n_estimators'] = 2000
    # _, oof_1, rmse, model1 = cv_pipeline(train_feats, test_feats, params, params['boosting_type'])

In [ ]:
# 0.602618 with feats_2 and vektorizer for 1,1 n-grams
# 0.602250 with feats_1 and vektorizer for 1,1 n-grams

In [9]:
print((f'The shape of train_feats is {train_feats.shape}'))

# lgb_params_3 = params_df.loc[1]['params']

_, oof_1, rmse, model1 = cv_pipeline(train_feats, test_feats, lgb_params_1, lgb_params_1['boosting_type'])
_, oof_2, rmse, model2 = cv_pipeline(train_feats, test_feats, lgb_params_2, lgb_params_2['boosting_type'])
_, oof_3, rmse, model3 = cv_pipeline(train_feats, test_feats, lgb_params_3, lgb_params_3['boosting_type'])

blend = pd.concat([oof_1, oof_2, oof_3], axis=0)
blend_scores = blend.groupby(['id','score'])['prediction'].mean().reset_index()
blend_rmse = mean_squared_error(blend_scores['score'], blend_scores['prediction'], squared=False)
print(f'Blend RMSE {blend_rmse}')

The shape of train_feats is (2471, 446)
LGBM Average RMSE over 50 folds: 0.602250
LGBM Average RMSE over 50 folds: 0.602758
LGBM Average RMSE over 50 folds: 0.603808
Blend RMSE 0.601391683184638


In [10]:
print((f'The shape of train_feats is {train_feats.shape}'))

# lgb_params_3 = params_df.loc[1]['params']

_, oof_1, rmse, model1 = cv_pipeline(train_feats, test_feats, lgb_params_1, lgb_params_1['boosting_type'])
_, oof_2, rmse, model2 = cv_pipeline(train_feats, test_feats, lgb_params_2, lgb_params_2['boosting_type'])

blend = pd.concat([oof_1, oof_2], axis=0)
blend_scores = blend.groupby(['id','score'])['prediction'].mean().reset_index()
blend_rmse = mean_squared_error(blend_scores['score'], blend_scores['prediction'], squared=False)
print(f'Blend RMSE {blend_rmse}')

The shape of train_feats is (2471, 446)
LGBM Average RMSE over 50 folds: 0.602250
LGBM Average RMSE over 50 folds: 0.602758
Blend RMSE 0.6014560217995425


New LGBM_params2 - Blend RMSE 0.602450743997947
Original 3 - Blend RMSE 0.602521633922091

Percentage removed 0.25 - Full pred 0.602333. Top pred 0.602194
Percentage removed 0.25 - Full pred 0.602161. Top pred 0.602109

feats_2 - blend LGBM 2 - 0.6026870286911109
feats_2 - blend LGBM 3 - 0.602521633922091

feats_1 - blend LGBM 3 - 0.6030926120372674

In [7]:
pct_to_remv = 0.8 
#Percentage removed 0.8 - Full pred 0.603179. Top pred 0.601455

params = [lgb_params_1, lgb_params_2, lgb_params_3]
test_ids = test_feats.id
test_bl_results, train_bl_results = pd.DataFrame(), pd.DataFrame()
blend_scores = pd.read_pickle('blend_scores_ft2.pkl')

for pct_to_remv in [0.80, 0.85, 0.95]: # 1.050000 BEST

    for i, p in enumerate(params):

        bal_scores = create_specific_balanced_datasets(train_scores, 
                                                        scores_to_split=[3, 3.5, 4, 4.5], 
                                                        pct_to_remv=pct_to_remv,
                                                        n_datasets=5,
                                                        seed=seed+i)
        
        for ds in bal_scores:
                    
                    ids = ds.id.unique()
                    test_preds, oof_results, rmse = cv_balanced_pipeline(train_feats=train_feats,
                                                                        test_feats=test_feats,
                                                                        lgb_params=p,
                                                                        balanced_dataset_ids=ids,
                                                                        boosting_type=p['boosting_type']
                                                                        )
                    
                    data = {'id': test_ids, 'prediction': test_preds}
                    test_tmp = pd.DataFrame(data=data)
                    test_bl_results = pd.concat([test_bl_results, test_tmp], axis=0)
                    train_bl_results = pd.concat([train_bl_results, oof_results], axis=0)

    train_avg_blc = train_bl_results.groupby(['id', 'score'])['prediction'].mean().reset_index()
    test_avg_blc = test_bl_results.groupby(['id'])['prediction'].mean().reset_index()

    train_bal_preds = train_avg_blc[(train_avg_blc['prediction'] < 2.5) | (train_avg_blc['prediction'] > 4.5)]
    train_bal_preds.to_pickle('bal_scores_ft2_full.pkl')
    train_concat_results = pd.concat([blend_scores, train_bal_preds], axis=0)
    train_blend_preds = train_concat_results.groupby(['id', 'score'])['prediction'].mean().reset_index()
    pred_full = np.sqrt(mean_squared_error(train_blend_preds['score'], train_blend_preds['prediction']))

    train_bal_preds = train_avg_blc[(train_avg_blc['prediction'] > 4.5)]
    train_bal_preds.to_pickle('bal_scores_ft2_top.pkl')
    train_concat_results = pd.concat([blend_scores, train_bal_preds], axis=0)
    train_blend_preds = train_concat_results.groupby(['id', 'score'])['prediction'].mean().reset_index()
    pred_top = np.sqrt(mean_squared_error(train_blend_preds['score'], train_blend_preds['prediction']))
    print(f'Percentage removed {pct_to_remv} - Full pred {pred_full:.6f}. Top pred {pred_top:.6f}')

LGBM Average RMSE over 50 folds: 0.649328
LGBM Average RMSE over 50 folds: 0.646151
LGBM Average RMSE over 50 folds: 0.650191
LGBM Average RMSE over 50 folds: 0.649058
LGBM Average RMSE over 50 folds: 0.645658
LGBM Average RMSE over 50 folds: 0.656797
LGBM Average RMSE over 50 folds: 0.652602
LGBM Average RMSE over 50 folds: 0.652666
LGBM Average RMSE over 50 folds: 0.650218
LGBM Average RMSE over 50 folds: 0.654699
LGBM Average RMSE over 50 folds: 0.651360
LGBM Average RMSE over 50 folds: 0.646451
LGBM Average RMSE over 50 folds: 0.649629
LGBM Average RMSE over 50 folds: 0.657803
LGBM Average RMSE over 50 folds: 0.649285
Percentage removed 0.8 - Full pred 0.603179. Top pred 0.601455
LGBM Average RMSE over 50 folds: 0.658478
LGBM Average RMSE over 50 folds: 0.657686
LGBM Average RMSE over 50 folds: 0.660616
LGBM Average RMSE over 50 folds: 0.662334
LGBM Average RMSE over 50 folds: 0.657820
LGBM Average RMSE over 50 folds: 0.666143
LGBM Average RMSE over 50 folds: 0.664742
LGBM Average 

##### Test one single instance of percentage removal

In [ ]:
test_preds_st, oof_results_st, rmse = cv_pipeline(train_feats, test_feats, lgb_params_1)
data = {'id': test_ids, 'prediction': test_preds_st}
test_tmp_st = pd.DataFrame(data=data)

train_bal_preds = train_avg_blc[(train_avg_blc['prediction'] < 2.5) | (train_avg_blc['prediction'] > 4.5)]
train_concat_results = pd.concat([oof_results_st, train_bal_preds], axis=0)
train_blend_preds = train_concat_results.groupby(['id', 'score'])['prediction'].mean().reset_index()
pred_full = np.sqrt(mean_squared_error(train_blend_preds['score'], train_blend_preds['prediction']))

train_bal_preds = train_avg_blc[(train_avg_blc['prediction'] > 4.5)]
train_concat_results = pd.concat([oof_results_st, train_bal_preds], axis=0)
train_blend_preds = train_concat_results.groupby(['id', 'score'])['prediction'].mean().reset_index()
pred_top = np.sqrt(mean_squared_error(train_blend_preds['score'], train_blend_preds['prediction']))
print(f'Full pred {pred_full}. Top pred {pred_top}')

LGBM Average RMSE over 50 folds: 0.603452


0.6027577504973198

In [30]:
train_bal_preds = train_avg_blc[(train_avg_blc['prediction'] < 2.5) | (train_avg_blc['prediction'] > 4.5)]
train_concat_results = pd.concat([oof_results_st, train_bal_preds], axis=0)
train_blend_preds = train_concat_results.groupby(['id', 'score'])['prediction'].mean().reset_index()
np.sqrt(mean_squared_error(train_blend_preds['score'], train_blend_preds['prediction']))

0.6031404980252844

In [24]:
train_bal_preds = train_avg_blc[(train_avg_blc['prediction'] > 4.5)]
train_concat_results = pd.concat([oof_results_st, train_bal_preds], axis=0)
train_blend_preds = train_concat_results.groupby(['id', 'score'])['prediction'].mean().reset_index()
np.sqrt(mean_squared_error(train_blend_preds['score'], train_blend_preds['prediction']))

0.60293358997378

In [ ]:
# COMPARE ALL FEATURES WITH BASELINE - ONE BY ONE
results = compare_with_baseline(base_dir=FEATURE_STORE, 
                                base_train_feats=base_train_feats,
                                base_test_feats=base_test_feats,
                                baseline_metrics=0.604499,
                                train_scores=train_scores)

['IKI', 'adj_eff_time', 'rep_cut', 'action_time_gap_by_acti', 'at_by_bucket', 'action_time_gap', 'wpm_feats', 'wc_chage']
IKI
LGBM Average RMSE over 50 folds: 0.603993
Features: IKI. RMSE: 0.603993, Improvement: 0.000506
adj_eff_time
LGBM Average RMSE over 50 folds: 0.605109
Features: adj_eff_time. RMSE: 0.605109, Improvement: -0.000610
rep_cut
LGBM Average RMSE over 50 folds: 0.606213
Features: rep_cut. RMSE: 0.606213, Improvement: -0.001714
action_time_gap_by_acti
LGBM Average RMSE over 50 folds: 0.604801
Features: action_time_gap_by_acti. RMSE: 0.604801, Improvement: -0.000302
at_by_bucket
LGBM Average RMSE over 50 folds: 0.605348
Features: at_by_bucket. RMSE: 0.605348, Improvement: -0.000849
action_time_gap
LGBM Average RMSE over 50 folds: 0.604974
Features: action_time_gap. RMSE: 0.604974, Improvement: -0.000475
wpm_feats
LGBM Average RMSE over 50 folds: 0.605128
Features: wpm_feats. RMSE: 0.605128, Improvement: -0.000629
wc_chage
LGBM Average RMSE over 50 folds: 0.604975
Features

In [ ]:
# LGBM Average RMSE over 50 folds: 0.603452
# Features: ('action_time_gap_by_acti', 'action_time_gap').
INPUT_DIR = 'kaggle/input/linking-writing-processes-to-writing-quality'
FEATURE_STORE = 'feature_store'
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_1.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_1.pkl')
train_feats = train_feats.merge(train_scores, on=['id'], how='left')
# Usage
max_combination_length = 8 # You can adjust this to test different combination lengths
results_comb = compare_feature_combinations(base_dir=FEATURE_STORE, 
                                       base_train_feats=train_feats,
                                       base_test_feats=test_feats,
                                       baseline_metrics=0.604322,
                                       max_combination_length=max_combination_length)

Number of combinations: 255
Feature set: ('action_time_gap_by_acti', 'IKI', 'wpm_feats', 'rep_cut', 'adj_eff_time', 'at_by_bucket', 'wc_chage', 'action_time_gap')
Base train size: (2471, 419)
Merging: action_time_gap_by_acti. Size: (2471, 67)
Size after the merge: (2471, 485)
Merging: IKI. Size: (2471, 5)
Size after the merge: (2471, 489)
Merging: wpm_feats. Size: (2471, 3)
Size after the merge: (2471, 492)
Merging: rep_cut. Size: (2470, 12)
Size after the merge: (2471, 503)
Merging: adj_eff_time. Size: (2471, 19)
Size after the merge: (2471, 521)
Merging: at_by_bucket. Size: (2471, 13)
Size after the merge: (2471, 533)
Merging: wc_chage. Size: (2471, 31)
Size after the merge: (2471, 563)
Merging: action_time_gap. Size: (2471, 12)
Size after the merge: (2471, 574)
(2471, 574) (3, 573)
LGBM Average RMSE over 50 folds: 0.605169
Features: ('action_time_gap_by_acti', 'IKI', 'wpm_feats', 'rep_cut', 'adj_eff_time', 'at_by_bucket', 'wc_chage', 'action_time_gap'). RMSE: 0.605169, Improvement: 

In [ ]:
 # base_train_2 = pd.read_pickle('feature_store/base_feats/train_base_feats_1.pkl')
 # tr_ft1 = pd.read_pickle('feature_store/train/train_wpm_feats.pkl')
 # tr_ft2 = pd.read_pickle('feature_store/train/train_action_time_gap_by_acti.pkl')
 # tr_ft3 = pd.read_pickle('feature_store/train/train_IKI.pkl')
 # base_train_2 = base_train_2.merge(tr_ft1, on=['id'], how='left')
 # base_train_2 = base_train_2.merge(tr_ft2, on=['id'], how='left')
 # base_train_2 = base_train_2.merge(tr_ft3, on=['id'], how='left')
 # 
 # base_test_2 = pd.read_pickle('feature_store/base_feats/test_base_feats_1.pkl')
 # ts_ft1 = pd.read_pickle('feature_store/test/test_wpm_feats.pkl')
 # ts_ft2 = pd.read_pickle('feature_store/test/test_action_time_gap_by_acti.pkl')
 # ts_ft3 = pd.read_pickle('feature_store/test/test_IKI.pkl')
 # base_test_2 = base_test_2.merge(ts_ft1, on=['id'], how='left')
 # base_test_2 = base_test_2.merge(ts_ft2, on=['id'], how='left')
 # base_test_2 = base_test_2.merge(ts_ft3, on=['id'], how='left')

# base_train_2.to_pickle('feature_store/base_feats/train_base_feats_2.pkl')
# base_test_2.to_pickle('feature_store/base_feats/test_base_feats_2.pkl')

#(wpm_feats, IKI, action_time_gap_by_acti)

In [ ]:
base_train_2 = pd.read_pickle('feature_store/base_feats/train_base_feats_1.pkl')
tr_ft1 = pd.read_pickle('feature_store/train/train_action_time_gap_by_acti.pkl')
tr_ft2 = pd.read_pickle('feature_store/train/train_action_time_gap.pkl')
base_train_2 = base_train_2.merge(tr_ft1, on=['id'], how='left')
base_train_2 = base_train_2.merge(tr_ft2, on=['id'], how='left')

base_test_2 = pd.read_pickle('feature_store/base_feats/test_base_feats_1.pkl')
ts_ft1 = pd.read_pickle('feature_store/test/test_action_time_gap_by_acti.pkl')
ts_ft2 = pd.read_pickle('feature_store/train/train_action_time_gap.pkl')
base_test_2 = base_test_2.merge(ts_ft1, on=['id'], how='left')
base_test_2 = base_test_2.merge(ts_ft2, on=['id'], how='left')

# base_train_2.to_pickle('feature_store/base_feats/train_base_feats_2.pkl')
# base_test_2.to_pickle('feature_store/base_feats/test_base_feats_2.pkl')
print(base_train_2.shape, base_test_2.shape)

(2471, 495) (3, 385)


#### RESULTS

In [ ]:
results_comb.sort_values(by='Metric').head(10)

,Feature Combination,Metric,Improvement
225,"(action_time_gap_by_acti, action_time_gap)",0.603452,0.000870
40,"(action_time_gap_by_acti, IKI, wpm_feats, rep_...",0.603481,0.000841
173,"(action_time_gap_by_acti, wpm_feats, action_ti...",0.603664,0.000658
11,"(action_time_gap_by_acti, IKI, wpm_feats, rep_...",0.603668,0.000654
131,"(IKI, wpm_feats, rep_cut, action_time_gap)",0.603718,0.000604
142,"(IKI, rep_cut, at_by_bucket, action_time_gap)",0.603842,0.000480
110,"(action_time_gap_by_acti, wpm_feats, rep_cut, ...",0.603907,0.000415
97,"(action_time_gap_by_acti, IKI, wpm_feats, acti...",0.603984,0.000338
219,"(action_time_gap_by_acti, IKI)",0.604015,0.000307
220,"(action_time_gap_by_acti, wpm_feats)",0.604043,0.000279
